In [4]:
!pip install pytesseract

In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import os
import cv2
import pytesseract
import easyocr
import re
import time

In [3]:
print(pytesseract.__version__)

0.3.10


In [ ]:

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

def display_images():
    result_window = tk.Toplevel(root)
    result_window.title("Detected Plates")

    canvas = tk.Canvas(result_window)
    scrollbar = tk.Scrollbar(result_window, orient="vertical", command=canvas.yview)
    scrollbar.pack(side="right", fill="y")
    canvas.pack(side="left", fill="both", expand=True)
    canvas.configure(yscrollcommand=scrollbar.set)

    frame_results = tk.Frame(canvas)
    canvas.create_window((0, 0), window=frame_results, anchor="nw")

    row_counter = 0

    for filename in os.listdir("carplates"):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join("carplates", filename)

            img = cv2.imread(image_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)

            reader = easyocr.Reader(['en'])
            result_easyocr = reader.readtext(blurred)

            result_pytesseract = pytesseract.image_to_string(gray, lang='eng')

            image = Image.open(image_path)
            photo = ImageTk.PhotoImage(image)
            label_image = tk.Label(frame_results, image=photo)
            label_image.image = photo
            label_image.grid(row=row_counter, column=0, padx=10, pady=10, sticky="nsew")

            text_frame_easyocr = tk.Frame(frame_results)
            text_frame_easyocr.grid(row=row_counter, column=1, padx=10, pady=10, sticky="nsew")
            tk.Label(text_frame_easyocr, text=f"Text in {filename} (EasyOCR):", font=("Arial", 12, "bold")).pack()
            for detection in result_easyocr:
                tk.Label(text_frame_easyocr, text=detection[1]).pack()

            text_frame_pytesseract = tk.Frame(frame_results)
            text_frame_pytesseract.grid(row=row_counter, column=2, padx=10, pady=10, sticky="nsew")
            tk.Label(text_frame_pytesseract, text=f"Text in {filename} (Pytesseract):", font=("Arial", 12, "bold")).pack()
            tk.Label(text_frame_pytesseract, text=result_pytesseract).pack()

            row_counter += 1

    canvas.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))


    
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_skip = max(total_frames // 10, 1)

    text_saved = [] 

    if not os.path.exists("carplates"):
        os.makedirs("carplates")

    result_window = tk.Toplevel(root)
    result_window.title("Results")

    text_display = tk.Text(result_window, height=10, width=50)
    text_display.pack()

    for frame_index in range(0, total_frames, frame_skip):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()

        if not ret:
            break

        height, width = frame.shape[:2]

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        canny = cv2.Canny(blurred, 120, 255, 1)
        cnts = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]

        for i, c in enumerate(cnts):
            x, y, w, h = cv2.boundingRect(c)
            if w >= 3 * h and w <= 6 * h and w <= width / 3 and w >= width / 15:
                config = "--psm 11 --oem 3 -c textord_tabfind_find_tables=0 textord_tabfind_only_strokewidths_below_height_fraction=0"
                roi = gray[y:y+h, x:x+w]
                text = pytesseract.image_to_string(roi, lang='eng', config=config)

                text = re.sub(r'\s+', '', text)
                text = re.sub(r'\W', '', text)

                if 7 <= len(text) <= 9:
                    print(f"Detected text: {text}")
                    text_saved.append(text) 
                    text_display.insert(tk.END, text + "\n") 

                    cv2.rectangle(frame, (x, y), (x + w, y + h), (36, 255, 12), 2)
                    cropped_region = frame[y:y+h, x:x+w]
                    cv2.imwrite(f"carplates/frame_{frame_index}_region_{i}.jpg", cropped_region)

        cv2.imwrite(f"frame_{frame_index}.jpg", frame)

        time.sleep(0.1)

    cap.release()

    print(f"All detected texts: {text_saved}")

    display_images()


    
def upload_picture():
    filename = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.jpeg")])
    if filename:
        img = cv2.imread(filename)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        result_window = tk.Toplevel(root)
        result_window.title("Detected Plate")

        reader = easyocr.Reader(['en'])
        result_easyocr = reader.readtext(blurred)

        result_pytesseract = pytesseract.image_to_string(blurred, lang='eng')

        image = Image.open(filename)
        photo = ImageTk.PhotoImage(image)
        label_image = tk.Label(result_window, image=photo)
        label_image.image = photo
        label_image.pack(padx=10, pady=10)

        tk.Label(result_window, text="Text Detected (EasyOCR):", font=("Arial", 12, "bold")).pack()
        for detection in result_easyocr:
            tk.Label(result_window, text=detection[1]).pack()

        tk.Label(result_window, text="Text Detected (Pytesseract):", font=("Arial", 12, "bold")).pack()
        tk.Label(result_window, text=result_pytesseract).pack()


def upload_video():
    filename = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi;*.mkv")])
    if filename:
        print("Video uploaded:", filename)
        process_video(filename)

def exit_program():
    root.destroy()

root = tk.Tk()
root.title("Menu")

window_width = 400
window_height = 200
root.geometry(f"{window_width}x{window_height}")

button_width = 20
button_height = 2
picture_button = tk.Button(root, text="Upload Picture", command=upload_picture, width=button_width, height=button_height)
video_button = tk.Button(root, text="Upload Video", command=upload_video, width=button_width, height=button_height)
exit_button = tk.Button(root, text="Exit", command=exit_program, width=button_width, height=button_height)

pad_y = 10
picture_button.pack(pady=pad_y)
video_button.pack(pady=pad_y)
exit_button.pack(pady=pad_y)

root.mainloop()


Video uploaded: C:/asdasd.mp4
Detected text: 990H7404
Detected text: ii8ss_re
Detected text: 90HzA01
Detected text: 990HZA01
Detected text: M29SOHZA


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


All detected texts: ['990H7404', 'ii8ss_re', '90HzA01', '990HZA01', 'M29SOHZA']


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster wi